In [ ]:
import os

In [ ]:
directory = "results/classifications/20210427/"

### Rename files

In [ ]:
# for file in os.listdir(directory):
#     #print(file[0:58]+'20201215.tif')
#     os.rename(directory+file, directory+file[0:58]+'20201215.tif')

## Mosiac tifs

In [ ]:
#build vrt
os.chdir(directory)
os.system("gdalbuildvrt " + "Eastern_gm_mads_two_seasons_20210427_mosaic.vrt *.tif")

#build mosaic from .vrt
os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       + "Eastern_gm_mads_two_seasons_20210427_mosaic.vrt " + "Eastern_gm_mads_two_seasons_20210427_mosaic.tif")

## Clip mosaic to AEZ

In [ ]:
# !pip install -e ../../production/dea_ml
# !pip install git+https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git@minty-fresh-sandbox#subdirectory=Tools

In [1]:
import xarray as xr
import numpy as np
import geopandas as gpd
from datacube.utils.cog import write_cog
import sys

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:44879 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 15 Memory: 104.37 GB


In [3]:
# aez_region = 'data/Eastern.shp'
aez_region = 'data/WDPA_eastern.shp'
pred_tif = "results/classifications/20210427/Eastern_gm_mads_two_seasons_20210427_mosaic_clipped.tif"
chunks={'x':10000, 'y':10000}

In [4]:
aez = gpd.read_file(aez_region)
da = xr.open_rasterio(pred_tif)

In [5]:
aez = xr_rasterize(aez, da)

Rasterizing to match xarray.DataArray dimensions (172800, 96000)


In [6]:
aez = 1-aez

In [8]:
da = da.chunk(chunks).where(aez.chunk(chunks)).astype(np.int8)

In [9]:
%%time
da = da.compute()

CPU times: user 18.3 s, sys: 50 s, total: 1min 8s
Wall time: 2min 39s


In [10]:
write_cog(da.squeeze(),
          pred_tif[:-4]+'_WDPA_masked.tif',
          overwrite=True)

PosixPath('results/classifications/20210427/Eastern_gm_mads_two_seasons_20210427_mosaic_clipped_WDPA_masked.tif')

In [ ]:
# write_cog(aez.squeeze(),
#           'data/WDPA_eastern.tif',
#           overwrite=True)